<a href="https://colab.research.google.com/github/lucdekeijzer/film_recommendations/blob/main/chatbot_translated_bugfixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colorama
!pip install scikit_learn
!pip install Flask
!pip install beautifulsoup4
!pip install bs4
!pip install googletrans==4.0.0rc1

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 9.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=f5a37eb8c67e1a9cc0f78e2758ea8bf1b0ae40acd1b9cfdec7c1666378ad5acc
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import googletrans
from googletrans import Translator
import requests
import pandas as pd


In [15]:
#with open('intents.json') as file:
    #data = json.load(file)

data = pd.read_json("https://raw.githubusercontent.com/lucdekeijzer/film_recommendations/main/intents.json")
 

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 153       
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
_________________________________________________________________


In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 8ms/step - loss: 2.1925 - accuracy: 0.1778
Epoch 2/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1896 - accuracy: 0.1778
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1872 - accuracy: 0.1778
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1845 - accuracy: 0.1778
Epoch 5/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1818 - accuracy: 0.1778
Epoch 6/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1795 - accuracy: 0.1778
Epoch 7/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1773 - accuracy: 0.1778
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1747 - accuracy: 0.1778
Epoch 9/500
2/2 [==============================] - 0s 5ms/step - loss: 2.1724 - accuracy: 0.1778
Epoch 10/500
2/2 [==============================] - 0s 5ms/step - loss: 2.1702 - accuracy: 0.1778
Epoch 11/500
2/2 [===========

In [8]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [9]:
def webscraper(x):
  from urllib.request import Request, urlopen
  from bs4 import BeautifulSoup
  import requests
  from random import randint

  #x = inp

  root = "https://www.google.com"
  link = "https://www.google.com/search?q=top+10+{}+movies&oq=top+&aqs=chrome.0.69i59l3j69i57j69i60l4.651j0j4&sourceid=chrome&ie=UTF-8".format(x)
  results = []

  req = Request(link, headers = {"User-Agent": "Mozilla/5.0"})
  webpage = urlopen(req).read()
  with requests.Session() as c:
    soup = BeautifulSoup(webpage, "html5lib")
    #print(soup)
    for item in soup.find_all('div', attrs = {"class": "RWuggc"}):
      results.append(item)

  titles = []
  for word in results:
    for j in word:
      for i in j:
        for p in i:
          titles.append(p)

  end_of_titles = len(titles)
  movie_index = randint(0, end_of_titles )
  movie_index_end = movie_index + 1
  return(titles[movie_index:movie_index_end])

In [13]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle
translator = Translator()

#with open("intents.json") as file:
#    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        language = (translator.detect(inp)).lang
        inp_translated = (translator.translate(inp)).text
        if inp_translated.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp_translated]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
              answer = np.random.choice(i['responses'])
              trans_answer = translator.translate(answer, dest = language).text
              print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)

        if tag == "thanks":
            answer = np.random.choice(i['responses'])
            trans_answer = translator.translate(answer, dest = language).text
            #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
            break
        if tag == "genres":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(webscraper(inp_translated))
          genre = inp_translated
        if tag == "another":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(webscraper(genre))
          



        #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: It's movie time
ChatBot: Any time!


In [ ]:
#print(googletrans.LANGUAGES)

example = "Hallo, hoe gaat het?"
transl_ex = (translator.translate(example)).text
print(transl_ex)


language = (translator.detect("Lorem Ipsum")).lang
print(language)

print((translator.translate(transl_ex, dest = language)).text)

#translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
#for translation in translations:
#print(translation.origin, ' -> ', translation.text)
# The quick brown fox  ->  빠른 갈색 여우
# jumps over  ->  이상 점프
# the lazy dog  ->  게으른 개

In [ ]:
titles = []
for word in results:
  for j in word:
    for i in j:
      for p in i:
        titles.append(p)

print(titles)
